### 유전체 정보 품종 분류 AI 경진대회
---
- 출처: https://dacon.io/competitions/official/236035/overview/description

In [1]:
# 모듈 생성
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import all_estimators
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

[1] 데이터 준비

In [3]:
# 데이터 로딩
train = pd.read_csv('./open/train.csv').iloc[:,1:]
test = pd.read_csv('./open/test.csv').iloc[:,1:]

In [4]:
train

,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
0,0,0,0,2,G G,A G,A A,G A,C A,A A,A A,G G,A A,G G,A G,A A,A A,A A,A A,B
1,0,0,0,2,A G,A G,C A,A A,A A,A G,A A,G A,A A,A G,A A,G A,G G,A A,A A,C
2,0,0,0,2,G G,G G,A A,G A,C C,G G,A A,G A,G A,A G,A A,A A,A A,A A,A A,B
3,0,0,0,1,A A,G G,A A,G A,A A,G G,G G,A A,G G,A G,G G,G G,G G,A A,G G,A
4,0,0,0,2,G G,G G,C C,A A,C C,A A,A A,A A,A A,G G,A A,A A,A G,A A,G A,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,0,0,0,2,A G,A G,A A,G A,C C,A G,A A,G A,A A,G G,A G,G A,A A,A A,A A,B
258,0,0,0,2,G G,A A,C A,A A,A A,A G,G A,G A,A A,A G,A G,A A,A G,A A,G A,C
259,0,0,0,1,A G,G G,A A,G A,A A,A G,G G,G A,G A,A A,G G,G G,G G,C A,G G,A
260,0,0,0,1,A A,G G,A A,G A,A A,G G,G G,A A,G A,A G,A G,G A,G G,C A,G G,A


In [5]:
test

,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
0,0,0,0,1,A G,G G,A A,G A,A A,A G,G G,G A,G A,A G,A G,G A,G G,C A,G A
1,0,0,0,2,G G,A G,C C,G G,C C,A A,A A,A A,A A,G G,A G,A A,A A,A A,A A
2,0,0,0,2,G G,A G,A A,A A,C A,A G,A A,A A,A A,A G,A A,G A,G G,A A,G G
3,0,0,0,2,G G,A G,C A,A A,C C,A A,A A,A A,A A,G G,A A,G A,A G,A A,A A
4,0,0,0,1,A A,G G,A A,G G,A A,G G,G G,A A,G G,A G,G G,G A,G G,A A,G G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,0,0,0,2,A G,G G,C C,A A,C A,A G,A A,G G,A A,G G,G G,A A,A A,A A,G A
171,0,0,0,2,G G,A A,A A,A A,C A,A G,A A,A A,A A,A G,A A,A A,A G,A A,G A
172,0,0,0,2,G G,A A,A A,A A,C A,A G,A A,A A,A A,G G,A G,A A,A G,A A,G G
173,0,0,0,2,A G,G G,C A,G A,C C,G G,A A,G A,A A,G G,A G,A A,A A,A A,A A


[1-1] 결측치 확인

In [6]:
train.isnull().sum()

father    0
mother    0
gender    0
trait     0
SNP_01    0
SNP_02    0
SNP_03    0
SNP_04    0
SNP_05    0
SNP_06    0
SNP_07    0
SNP_08    0
SNP_09    0
SNP_10    0
SNP_11    0
SNP_12    0
SNP_13    0
SNP_14    0
SNP_15    0
class     0
dtype: int64

In [7]:
test.isnull().sum()

father    0
mother    0
gender    0
trait     0
SNP_01    0
SNP_02    0
SNP_03    0
SNP_04    0
SNP_05    0
SNP_06    0
SNP_07    0
SNP_08    0
SNP_09    0
SNP_10    0
SNP_11    0
SNP_12    0
SNP_13    0
SNP_14    0
SNP_15    0
dtype: int64

[1-2] 중복 확인

In [8]:
train.duplicated().sum()

6

In [9]:
train = train.drop_duplicates(keep='last')

In [10]:
test.duplicated().sum()

0

[1-3] 라벨 분포

In [11]:
data = train.iloc[:,:-1]
target = train.iloc[:,-1]

In [12]:
target.value_counts()

B    113
C     79
A     64
Name: class, dtype: int64

[1-4] 라벨 인코더

In [13]:
le = LabelEncoder()

for obj in data.dtypes.index:
    if data[obj].dtype == 'object':
        data[obj] = le.fit_transform(data[obj])
        test[obj] = le.transform(test[obj])

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 256 entries, 0 to 261
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   father  256 non-null    int64
 1   mother  256 non-null    int64
 2   gender  256 non-null    int64
 3   trait   256 non-null    int64
 4   SNP_01  256 non-null    int32
 5   SNP_02  256 non-null    int32
 6   SNP_03  256 non-null    int32
 7   SNP_04  256 non-null    int32
 8   SNP_05  256 non-null    int32
 9   SNP_06  256 non-null    int32
 10  SNP_07  256 non-null    int32
 11  SNP_08  256 non-null    int32
 12  SNP_09  256 non-null    int32
 13  SNP_10  256 non-null    int32
 14  SNP_11  256 non-null    int32
 15  SNP_12  256 non-null    int32
 16  SNP_13  256 non-null    int32
 17  SNP_14  256 non-null    int32
 18  SNP_15  256 non-null    int32
dtypes: int32(15), int64(4)
memory usage: 25.0 KB


In [15]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   father  175 non-null    int64
 1   mother  175 non-null    int64
 2   gender  175 non-null    int64
 3   trait   175 non-null    int64
 4   SNP_01  175 non-null    int32
 5   SNP_02  175 non-null    int32
 6   SNP_03  175 non-null    int32
 7   SNP_04  175 non-null    int32
 8   SNP_05  175 non-null    int32
 9   SNP_06  175 non-null    int32
 10  SNP_07  175 non-null    int32
 11  SNP_08  175 non-null    int32
 12  SNP_09  175 non-null    int32
 13  SNP_10  175 non-null    int32
 14  SNP_11  175 non-null    int32
 15  SNP_12  175 non-null    int32
 16  SNP_13  175 non-null    int32
 17  SNP_14  175 non-null    int32
 18  SNP_15  175 non-null    int32
dtypes: int32(15), int64(4)
memory usage: 15.8 KB


In [16]:
data = data.iloc[:,3:]
test = test.iloc[:,3:]

[1-5] 상관관계

In [17]:
data.corr()

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
trait,1.000000,0.716552,-0.548963,0.557682,-0.588362,0.531760,-0.506438,-0.868286,0.600234,-0.671703,0.656018,-0.600118,-0.747515,-0.549843,-0.712081,-0.603790
SNP_01,0.716552,1.000000,-0.350355,0.456594,-0.356465,0.426311,-0.404925,-0.602713,0.443588,-0.425808,0.499194,-0.366410,-0.545168,-0.369487,-0.547601,-0.435068
SNP_02,-0.548963,-0.350355,1.000000,-0.247910,0.388995,-0.103615,0.349158,0.465850,-0.289388,0.395717,-0.255365,0.388122,0.451491,0.158334,0.337866,0.333065
SNP_03,0.557682,0.456594,-0.247910,1.000000,-0.326283,0.265431,-0.330986,-0.505565,0.518971,-0.402002,0.465165,-0.298406,-0.346265,-0.259342,-0.465705,-0.284921
SNP_04,-0.588362,-0.356465,0.388995,-0.326283,1.000000,-0.239664,0.442624,0.536645,-0.273331,0.497789,-0.285784,0.389643,0.394806,0.249901,0.417294,0.274909
SNP_05,0.531760,0.426311,-0.103615,0.265431,-0.239664,1.000000,-0.316265,-0.582536,0.289494,-0.319186,0.446362,-0.145759,-0.386329,-0.513056,-0.407150,-0.346180
SNP_06,-0.506438,-0.404925,0.349158,-0.330986,0.442624,-0.316265,1.000000,0.447116,-0.380604,0.469652,-0.378767,0.363032,0.345745,0.326628,0.347069,0.342826
SNP_07,-0.868286,-0.602713,0.465850,-0.505565,0.536645,-0.582536,0.447116,1.000000,-0.475924,0.615494,-0.563812,0.516135,0.631346,0.480518,0.575225,0.542650
SNP_08,0.600234,0.443588,-0.289388,0.518971,-0.273331,0.289494,-0.380604,-0.475924,1.000000,-0.469325,0.574004,-0.230273,-0.415749,-0.401596,-0.550648,-0.364378
SNP_09,-0.671703,-0.425808,0.395717,-0.402002,0.497789,-0.319186,0.469652,0.615494,-0.469325,1.000000,-0.429569,0.414604,0.479003,0.333354,0.441094,0.339723


[2] 학습/테스트 데이터 분리

In [18]:
X_train, X_test, y_train, y_test = train_test_split(data, 
                                                    target,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=target)

In [19]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((204, 16), (52, 16), (204,), (52,))

[3] 최적의 모델 찾기

In [20]:
models = all_estimators(type_filter='classifier')
scores = []
names = []

for name, model in models:
    try:
        # 모델 객체 생성
        md = model()
        # 학습
        md.fit(X_train, y_train)
        # 평가
        result1 = md.score(X_train, y_train)
        result2 = md.score(X_test, y_test)
        names.append(name)
        scores.append((name, np.round(result1,3), np.round(result2,3)))
    except:
        pass

In [21]:
scoresDf = pd.DataFrame(scores, columns=['model', 'train_score', 'test_score'])
scoresDf.sort_values(by='test_score', ascending=False).head(15)

,model,train_score,test_score
32,SVC,0.990,0.942
9,ExtraTreesClassifier,1.000,0.923
30,RidgeClassifierCV,0.980,0.923
29,RidgeClassifier,0.971,0.923
28,RandomForestClassifier,1.000,0.923
24,NuSVC,0.980,0.923
17,LinearDiscriminantAnalysis,0.975,0.923
15,LabelPropagation,1.000,0.923
13,HistGradientBoostingClassifier,1.000,0.923
16,LabelSpreading,1.000,0.923


---

- SVC

[4] 모델 학습

In [22]:
from sklearn.svm import SVC

In [23]:
model = SVC(
        C=1.0,
        kernel='rbf',
        degree=3,
        gamma='scale',
        coef0=0.0,
        shrinking=True,
        probability=False,
        tol=0.001,
        cache_size=200,
        class_weight=None,
        verbose=False,
        max_iter=-1,
        decision_function_shape='ovr',
        break_ties=False,
        random_state=None)

# 학습하기
model.fit(X_train, y_train)

SVC()

[5] 모델 평가

In [24]:
from sklearn.metrics import accuracy_score

In [25]:
pred = model.predict(X_test)

In [26]:
accuracy_score(y_test, pred)

0.9423076923076923

- Light GBM

In [37]:
from lightgbm import LGBMClassifier

model2 = LGBMClassifier(n_estimators=300,
                        max_depth=2,
                        learning_rate=0.05,
                        random_state=22)
model2.fit(X_train, y_train)

LGBMClassifier(learning_rate=0.05, max_depth=2, n_estimators=300,
               random_state=22)

In [38]:
pred2 = model2.predict(X_test)

In [39]:
accuracy_score(y_test, pred2)

0.9230769230769231

[6] 모델 예측 - test.csv

In [40]:
predict = model2.predict(test)

In [41]:
sub = pd.read_csv('./open/sample_submission.csv')
sub.head()

,id,class
0,TEST_000,A
1,TEST_001,A
2,TEST_002,A
3,TEST_003,A
4,TEST_004,A


In [42]:
sub['class'] = predict

In [43]:
sub.head()

,id,class
0,TEST_000,A
1,TEST_001,B
2,TEST_002,C
3,TEST_003,B
4,TEST_004,A


In [44]:
sub.to_csv('제출용.csv', index=False)

In [45]:
# 확인
pd.read_csv('제출용.csv')

,id,class
0,TEST_000,A
1,TEST_001,B
2,TEST_002,C
3,TEST_003,B
4,TEST_004,A
...,...,...
170,TEST_170,B
171,TEST_171,C
172,TEST_172,C
173,TEST_173,B
